## Import

In [1]:
import beamtest_analysis_helper as helper
from pathlib import Path
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm
import hist

In [2]:
chip_labels = [0, 1, 2, 3]
chip_names = ["ET2_EPIR_Pair1", "ET2_BAR_4", "ET2_BAR_6", "ET2_CNM_1-3"]
offsets = [15, 5, 15, 5]
high_voltages = [260, 260, 260, 200]
board_to_analyze = [0,1,2]
ignore_boards = [3]

run_name = "run13"

chip_fignames = chip_names
chip_figtitles = [
    f"(Trigger) EPIR Pair1 HV{high_voltages[0]}V OS:{offsets[0]}",
    f"(DUT1) Barcelona 4 HV{high_voltages[1]}V OS:{offsets[1]}",
    f"(Reference) Barcelona 6 HV{high_voltages[2]}V OS:{offsets[2]}",
    f"(DUT2) CNM (HPK Sensor) 1-3 HV{high_voltages[3]}V OS:{offsets[3]}"]

## Load tracks

In [ ]:
track_df = pd.read_csv(f"./{run_name}_good_track_candidates.csv")

## Filtering data based on track

In [ ]:
files = glob(f'./desy_TB_{run_name}/*.feather')

last_evt = 0
dataframes = []
last_idx = -1
tmp_df = None
min_file_count = 8
track_pivots = {i:[] for i in range(len(track_df))}

for idx, ifile in enumerate(tqdm(files)):

    ## Skip all files once they've already been accomodated in a previous batch
    if(last_idx==-2):
        continue

    ## load file and alter evt
    run_df = pd.read_feather(ifile)
    run_df.drop(columns=['evt_number', 'bcid', 'l1a_counter', 'ea'], inplace=True)

    if idx > 0:
        run_df['evt'] += last_evt
    last_evt += run_df['evt'].unique()[-1]

    ## Check if this is the first file of the batch
    if last_idx == -1:
        tmp_df = run_df.copy()
        last_idx = idx
        continue

    ## Check if the current file falls into the current batch but doesn't complete it
    elif idx-last_idx+1<min_file_count:
        tmp_df = pd.concat([tmp_df, run_df])
        continue

    ## Check if the current file completes the current batch and pass the batch on
    else:
        tmp_df = pd.concat([tmp_df, run_df])
        last_idx = idx

        ## Check if there are files that don't complete a batch leftover after this file
        if((len(files)-last_idx-1<min_file_count)and(len(files)-last_idx-1>0)):
            for idx_inner, innerfile in enumerate(files):
                if(idx_inner<=last_idx): continue
                run_df = pd.read_feather(innerfile)
                run_df.drop(columns=['evt_number', 'bcid', 'l1a_counter', 'ea'], inplace=True)
                if idx_inner > 0:
                    run_df['evt'] += last_evt
                last_evt += run_df['evt'].unique()[-1]
                tmp_df = pd.concat([tmp_df, run_df])

            ## Flag to skip all these files that have already been taken into account
            last_idx = -2
            print("Working on an extended batch of files")

        ## Completed a batch nominally and now passing it on
        else:
            last_idx = -1
            print(f"Working on a clean batch of {min_file_count} files")

    ## Restrict to events with only one hit on the boards of interest
    tmp_df = helper.singlehit_event_clear(tmp_df, ignore_boards=ignore_boards)

    for i in tqdm(range(len(track_df))):

        ## Filter only the pixels of interest, dropping other hits on the boards of interest as well as boards not of interest
        pix_dict = {}

        for idx in board_to_analyze:
            pix_dict[idx] = [track_df.iloc[i][f'row_{idx}'], track_df.iloc[i][f'col_{idx}']]

        track_tmp_df = helper.pixel_filter(tmp_df, pix_dict)

        ## Selecting good hits with TDC cuts
        tdc_cuts = {}
        for idx in board_to_analyze:
            # board ID: [CAL LB, CAL UB, TOA LB, TOA UB, TOT LB, TOT UB]
            if idx == 0:
                tdc_cuts[idx] = [track_tmp_df.loc[track_tmp_df['board'] == idx]['cal'].mode()[0]-3, track_tmp_df.loc[track_tmp_df['board'] == idx]['cal'].mode()[0]+3,  100, 500, 0, 600]
            else:
                tdc_cuts[idx] = [track_tmp_df.loc[track_tmp_df['board'] == idx]['cal'].mode()[0]-3, track_tmp_df.loc[track_tmp_df['board'] == idx]['cal'].mode()[0]+3,  0, 1100, 0, 600]
        track_tmp_df = helper.tdc_event_selection(track_tmp_df, tdc_cuts_dict=tdc_cuts)

        ## Restrict to events with only one hit on the boards of interest, needed again as we may have dropped hits
        track_tmp_df = helper.singlehit_event_clear(track_tmp_df, ignore_boards=ignore_boards)

        ## Pivot Table to make tracks
        pivot_table = track_tmp_df.pivot(index=["evt"], columns=["board"], values=["row", "col", "toa", "tot", "cal"])
        track_pivots[i].append(pivot_table)

    del tmp_df, track_tmp_df, pivot_table

In [ ]:
track_pivots_concat = { i: pd.concat(track_pivots[i]) for i in range(len(track_df)) }

In [ ]:
len_list = []
table_dir = Path(f"./{run_name}_track_pivot_tables/").mkdir(parents=True, exist_ok=True)

for key,val in tqdm(track_pivots_concat.items()):
    len_list += [len(val)]
    val.reset_index(drop=True,inplace=True)
    val.to_csv(f"./{run_name}_track_pivot_tables/track_{key}.csv",index=False)

print(np.amax(len_list), np.amin(len_list))
del len_list

## Run Bootstrapping!

In [ ]:
final_dict = {}

for idx in board_to_analyze:
    final_dict[f'row{idx}'] = []
    final_dict[f'col{idx}'] = []
    final_dict[f'res{idx}'] = []
    final_dict[f'err{idx}'] = []

for ikey, itable in tqdm(track_pivots_concat.items()):

    sum_arr = {}
    sum_square_arr = {}
    iteration = 100
    sampling_fraction = 0.75
    counter = 0

    for idx in board_to_analyze:
        sum_arr[idx] = 0
        sum_square_arr[idx] = 0

    for iloop in tqdm(range(iteration)):

        tdc_filtered_df = itable.reset_index()

        n = int(sampling_fraction*tdc_filtered_df.shape[0])
        indices = np.random.choice(tdc_filtered_df['evt'].unique(), n, replace=False)
        tdc_filtered_df = tdc_filtered_df.loc[tdc_filtered_df['evt'].isin(indices)]

        if tdc_filtered_df.shape[0] < iteration/(3.*(1-sampling_fraction)):
            print('Warning!! Sampling size is too small. Skipping this track')
            break

        d = {
            'evt': tdc_filtered_df['evt'].unique(),
        }

        for idx in board_to_analyze:
            bins = 3.125/tdc_filtered_df['cal'][idx].mean()
            d[f'toa_b{str(idx)}'] = 12.5 - tdc_filtered_df['toa'][idx] * bins
            d[f'tot_b{str(idx)}'] = (2*tdc_filtered_df['tot'][idx] - np.floor(tdc_filtered_df['tot'][idx]/32)) * bins

        df_in_time = pd.DataFrame(data=d)
        del d, tdc_filtered_df

        corr_toas = helper.three_board_iterative_timewalk_correction(df_in_time, 5, 3, board_list=board_to_analyze)

        diffs = {}
        for board_a in board_to_analyze:
            for board_b in board_to_analyze:
                if board_b <= board_a:
                    continue
                name = f"{board_a}{board_b}"
                diffs[name] = np.asarray(corr_toas[f'toa_b{board_a}'] - corr_toas[f'toa_b{board_b}'])

        hists = {}
        for key in diffs.keys():
            hists[key] = hist.Hist(hist.axis.Regular(80, -1.2, 1.2, name="TWC_delta_TOA", label=r'Time Walk Corrected $\Delta$TOA [ns]'))
            hists[key].fill(diffs[key])

        try:
            fit_params_lmfit = {}
            for key in hists.keys():
                params = helper.lmfit_gaussfit_with_pulls(diffs[key], hists[key], std_range_cut=0.4, width_factor=1.25, fig_title='',
                                                    use_pred_uncert=True, no_show_fit=False, no_draw=True, get_chisqure=False)
                fit_params_lmfit[key] = params
            del params, hists, diffs, corr_toas

            resolutions = helper.return_resolution_three_board(fit_params_lmfit, var=list(fit_params_lmfit.keys()), board_list=board_to_analyze)

            if any(np.isnan(val) for key, val in resolutions.items()):
                print('fit results is not good, skipping this iteration')
                continue

            for key in resolutions.keys():
                sum_arr[key] += resolutions[key]
                sum_square_arr[key] += resolutions[key]**2

            counter += 1

        except:
            print('Failed, skipping')
            del hists, diffs, corr_toas

    if counter != 0:
        for idx in board_to_analyze:
            final_dict[f'row{idx}'].append(itable['row'][idx].unique()[0])
            final_dict[f'col{idx}'].append(itable['col'][idx].unique()[0])

        for key in sum_arr.keys():
            mean = sum_arr[key]/counter
            std = np.sqrt((1/(counter-1))*(sum_square_arr[key]-counter*(mean**2)))
            final_dict[f'res{key}'].append(mean)
            final_dict[f'err{key}'].append(std)
    else:
        print('Track is not validate for bootstrapping')

In [ ]:
final_df = pd.DataFrame(final_dict)
final_df.info()

In [ ]:
csv_tag = "_trigTOA100_500_oneHitTrigDut1Ref"
final_df.to_csv(f'./{run_name}{csv_tag}_resolutions.csv', index=False)

## Draw final resolution plots

In [ ]:
helper.plot_resolution_with_pulls(final_df, chipLabels=board_to_analyze, fig_title=chip_figtitles, fig_tag="", hist_bins=15)

In [ ]:
helper.plot_resolution_table(final_df, chipLabels=board_to_analyze, fig_title=chip_figtitles, fig_tag="", slides_friendly=True)